# Buffer
Replay Buffer is a very common module in DRL implementations. In Tianshou, the Buffer module can be viewed as a specialized form of Batch, designed to track all data trajectories and offering utilities like sampling methods beyond basic storage.

There are many kinds of Buffer modules in Tianshou, two most basic ones are ReplayBuffer and VectorReplayBuffer. The later one is specially designed for parallelized environments (will introduce in tutorial [Vectorized Environment](https://tianshou.readthedocs.io/en/master/02_notebooks/L3_Vectorized__Environment.html)). In this tutorial, we will focus on ReplayBuffer.

In [ ]:
import pickle

import numpy as np

from tianshou.data import Batch, ReplayBuffer

## Usages

### Basic usages as a batch
Typically, a buffer stores all data in batches, employing a circular-queue mechanism.

In [ ]:
# a buffer is initialised with its maxsize set to 10 (older data will be discarded if more data flow in).
print("========================================")
dummy_buf = ReplayBuffer(size=10)
print(dummy_buf)
print(f"maxsize: {dummy_buf.maxsize}, data length: {len(dummy_buf)}")

# add 3 steps of data into ReplayBuffer sequentially
print("========================================")
for i in range(3):
    dummy_buf.add(
        Batch(obs=i, act=i, rew=i, terminated=0, truncated=0, done=0, obs_next=i + 1, info={}),
    )
print(dummy_buf)
print(f"maxsize: {dummy_buf.maxsize}, data length: {len(dummy_buf)}")

# add another 10 steps of data into ReplayBuffer sequentially
print("========================================")
for i in range(3, 13):
    dummy_buf.add(
        Batch(obs=i, act=i, rew=i, terminated=0, truncated=0, done=0, obs_next=i + 1, info={}),
    )
print(dummy_buf)
print(f"maxsize: {dummy_buf.maxsize}, data length: {len(dummy_buf)}")

Just like Batch, ReplayBuffer supports concatenation, splitting, advanced slicing and indexing, etc.

In [ ]:
print(dummy_buf[-1])
print(dummy_buf[-3:])
# Try more methods you find useful in Batch yourself.

ReplayBuffer can also be saved into local disk, still keeping track of the trajectories. This is extremely helpful in offline DRL settings.

In [ ]:
_dummy_buf = pickle.loads(pickle.dumps(dummy_buf))

### Understanding reserved keys for buffer
As explained above, ReplayBuffer is specially designed to utilize the implementations of DRL algorithms. So, for convenience, we reserve certain nine reserved keys in Batch.

*   `obs`
*   `act`
*   `rew`
*   `terminated`
*   `truncated`
*   `done`
*   `obs_next`
*   `info`
*   `policy`

The meaning of these nine reserved keys are consistent with the meaning in [Gymansium](https://gymnasium.farama.org/index.html#). We would recommend you simply use these nine keys when adding batched data into ReplayBuffer, because
some of them are tracked in ReplayBuffer (e.g. "done" value is tracked to help us determine a trajectory's start index and end index, together with its total reward and episode length.)

```
buf.add(Batch(......, extro_info=0)) # This is okay but not recommended.
buf.add(Batch(......, info={"extro_info":0})) # Recommended.
```


### Data sampling
The primary purpose of maintaining a replay buffer in DRL is to sample data for training. `ReplayBuffer.sample()` and `ReplayBuffer.split(..., shuffle=True)` can both fulfill this need.

In [ ]:
dummy_buf.sample(batch_size=5)

## Trajectory tracking
Compared to Batch, a unique feature of ReplayBuffer is that it can help you track the environment trajectories.

First, let us simulate a situation, where we add three trajectories into the buffer. The last trajectory is still not finished yet.

In [ ]:
trajectory_buffer = ReplayBuffer(size=10)
# Add the first trajectory (length is 3) into ReplayBuffer
print("========================================")
for i in range(3):
    result = trajectory_buffer.add(
        Batch(
            obs=i,
            act=i,
            rew=i,
            terminated=1 if i == 2 else 0,
            truncated=0,
            done=i == 2,
            obs_next=i + 1,
            info={},
        ),
    )
    print(result)
print(trajectory_buffer)
print(f"maxsize: {trajectory_buffer.maxsize}, data length: {len(trajectory_buffer)}")

# Add the second trajectory (length is 5) into ReplayBuffer
print("========================================")
for i in range(3, 8):
    result = trajectory_buffer.add(
        Batch(
            obs=i,
            act=i,
            rew=i,
            terminated=1 if i == 7 else 0,
            truncated=0,
            done=i == 7,
            obs_next=i + 1,
            info={},
        ),
    )
    print(result)
print(trajectory_buffer)
print(f"maxsize: {trajectory_buffer.maxsize}, data length: {len(trajectory_buffer)}")

# Add the third trajectory (length is 5, still not finished) into ReplayBuffer
print("========================================")
for i in range(8, 13):
    result = trajectory_buffer.add(
        Batch(obs=i, act=i, rew=i, terminated=0, truncated=0, done=False, obs_next=i + 1, info={}),
    )
    print(result)
print(trajectory_buffer)
print(f"maxsize: {trajectory_buffer.maxsize}, data length: {len(trajectory_buffer)}")

### Episode length and rewards tracking
Notice that `ReplayBuffer.add()` returns a tuple of 4 numbers every time, meaning `(current_index, episode_reward, episode_length, episode_start_index)`. `episode_reward` and `episode_length` are valid only when a trajectory is finished. This might save developers some trouble.



### Episode index management
In the ReplayBuffer above, we can get access to any data step by indexing.


In [ ]:
print(trajectory_buffer)
print("========================================")

data = trajectory_buffer[6]
print(data)

We know that step "6" is not the start of an episode - which should be step "3", since "3-7" is the second trajectory we add into the ReplayBuffer - but we wonder how do we get the earliest index of that episode.

This may seem easy but actually it is not. We cannot simply look at the "done" flag preceding the start of a new episode, because since the third-added trajectory is not finished yet, step "3" is surrounded by flag "False". There are many things to consider. Things could get more nasty when using more advanced ReplayBuffer like VectorReplayBuffer, since it does not store the data in a simple circular-queue.

Luckily, all ReplayBuffer instances help you identify step indexes through a unified API. One can simply input an array of indexes and look for their previous index in the episode.

In [ ]:
# previous step of indexes [0, 1, 2, 3, 4, 5, 6] are:
print(trajectory_buffer.prev(np.array([0, 1, 2, 3, 4, 5, 6])))

Using `ReplayBuffer.prev()`, we know that the earliest step of that episode is step "3". Similarly, `ReplayBuffer.next()` helps us identify the last index of an episode regardless of which kind of ReplayBuffer we are using.

In [ ]:
# next step of indexes [4,5,6,7,8,9] are:
print(trajectory_buffer.next(np.array([4, 5, 6, 7, 8, 9])))

We can also search for the indexes which are labeled "done: False", but are the last step in a trajectory.

In [ ]:
print(trajectory_buffer.unfinished_index())

Aforementioned APIs will be helpful when we calculate quantities like GAE and n-step-returns in DRL algorithms ([Example usage in Tianshou](https://github.com/thu-ml/tianshou/blob/6fc68578127387522424460790cbcb32a2bd43c4/tianshou/policy/base.py#L384)). The unified APIs ensure a modular design and a flexible interface.

## Further Reading
### Other Buffer Module

*   PrioritizedReplayBuffer, which helps you implement [prioritized experience replay](https://arxiv.org/abs/1511.05952)
*   CachedReplayBuffer, one main buffer with several cached buffers (higher sample efficiency in some scenarios)
*   ReplayBufferManager, A base class that can be inherited (may help you manage multiple buffers).

Refer to the documentation and source code for further details.

### Support for steps stacking to use RNN in DRL.
There is an option called `stack_num` (default to 1) when initializing the ReplayBuffer, which may help you use RNN in your algorithm. Check the documentation for details.